# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    # select 2018 data
    nhl_df = nhl_df[nhl_df['year']==2018]
    # remove *
    nhl_df['team'] = nhl_df['team'].map(lambda x: x.strip('*'))
    # remove irrelevant rows
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Atlantic Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Metropolitan Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Central Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Pacific Division'].index, inplace=True)

    # create new column, city, and rename the column
    nhl_df['city'] = nhl_df['team'].apply(lambda x: x.split()[0])
    nhl_df['city'].replace({'New': 'New York City', 'St.': 'St. Louis', 'Vegas': 'Las Vegas', 'San': 'San Francisco Bay Area',
                       'Los': 'Los Angeles', 'Tempa': 'Tampa Bay Area', 'Florida': 'Miami–Fort Lauderdale', 'Washington': 'Washington, D.C.',
                       'Carolina': 'Raleigh', 'Minnesota': 'Minneapolis–Saint Paul', 'Colorado': 'Denver', 'Dallas': 'Dallas–Fort Worth',
                       'Anaheim': 'Los Angeles', 'Arizona': 'Phoenix', 'Tampa': 'Tampa Bay Area'}, inplace=True)
    # reformat data type for column, W, L, Population
    nhl_df['W'] = pd.to_numeric(nhl_df['W'])
    nhl_df['L'] = pd.to_numeric(nhl_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nhl_df, cities, left_on='city', right_on='Metropolitan area')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    population_by_region = df_new_group['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df_new_group['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    #raise NotImplementedError()
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [241]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    # select 2018 data
    nba_df = nba_df[nba_df['year']==2018]
    # remove irrelevant rows
    nba_df.drop(nba_df[nba_df['team'] == 'Atlantic Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Southeast Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Central Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Southeast Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Pacific Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Southwest Division'].index, inplace=True)

    # create new column, city, and rename the column
    nba_df['city'] = nba_df['team'].apply(lambda x: x.split()[0])
    # replace city name "new"
    nba_df['city'].replace({'San': 'San Antonio', 'Los': 'Los Angeles', 'Dallas': 'Dallas–Fort Worth', 'Indiana': 'Indianapolis',
                       'Miami': 'Miami–Fort Lauderdale', 'Washington': 'Washington, D.C.', 'Brooklyn': 'New York City',
                       'Golden': 'San Francisco Bay Area', 'Oklahoma': 'Oklahoma City', 'Utah': 'Salt Lake City', 'Minnesota': 'Minneapolis–Saint Paul',
                       'NewYorkCity': 'New York City', 'NewOrleans': 'New Orleans'}, inplace=True)
    nba_df['city'][10] = 'New York City'
    nba_df['city'][20] = 'New Orleans'
    # reformat data type for column, W, L, Population
    nba_df['W'] = pd.to_numeric(nba_df['W'])
    nba_df['L'] = pd.to_numeric(nba_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nba_df, cities, left_on='city', right_on='Metropolitan area', how='outer')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    # raise NotImplementedError()
    
    population_by_region = df_new_group['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df_new_group['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [136]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # YOUR CODE HERE
    # select 2018 data
    mlb_df = mlb_df[mlb_df['year']==2018]
    # create new column, city, and rename the column
    mlb_df['city'] = mlb_df['team'].apply(lambda x: x.split()[0])
    # rename city name 'San'
    mlb_df.loc[mlb_df['city'] == 'San', 'city'] = ['San Francisco Bay Area', 'San Diego']
    mlb_df['city'].replace({'New': 'New York City', 'Los': 'Los Angeles', 'St.': 'St. Louis',
                       'Tampa': 'Tampa Bay Area', 'Minnesota': 'Minneapolis–Saint Paul', 'Kansas': 'Kansas City',
                       'Texas': 'Dallas–Fort Worth', 'Washington': 'Washington, D.C.', 'Miami': 'Miami–Fort Lauderdale',
                       'Colorado': 'Denver', 'Arizona': 'Phoenix', 'Oakland': 'San Francisco Bay Area'}, inplace=True)
    # reformat data type for column, W, L, Population
    mlb_df['W'] = pd.to_numeric(mlb_df['W'])
    mlb_df['L'] = pd.to_numeric(mlb_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(mlb_df, cities, left_on='city', right_on='Metropolitan area', how='outer')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    # raise NotImplementedError()
  
    population_by_region = df_new_group['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df_new_group['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [166]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # YOUR CODE HERE
    # select 2018 data
    nfl_df = nfl_df[nfl_df['year']==2018]
    # remove irrelevant rows
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC East'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC North'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC South'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC West'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC East'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC North'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC South'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC West'].index, inplace=True)
    # create new column, city, and rename the column
    nfl_df['city'] = nfl_df['team'].apply(lambda x: x.split()[0])
    # rename city name "New"
    nfl_df.loc[nfl_df['city']=='New', 'city'] = ['Boston', 'New York City', 'New York City', 'New Orleans']
    nfl_df['city'].replace({'Miami': 'Miami–Fort Lauderdale', 'Tennessee': 'Nashville', 'Kansas': 'Kansas City',
                       'Los': 'Los Angeles', 'Oakland': 'San Francisco Bay Area', 'Dallas': 'Dallas–Fort Worth',
                       'Washington': 'Washington, D.C.', 'Minnesota': 'Minneapolis–Saint Paul', 'Green': 'Green Bay',
                       'Carolina': 'Charlotte', 'Tampa': 'Tampa Bay Area', 'San': 'San Francisco Bay Area',
                       'Arizona': 'Phoenix'}, inplace=True)
    # reformat data type for column, W, L, Population
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nfl_df, cities, left_on='city', right_on='Metropolitan area', how='outer')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    # raise NotImplementedError()
  
    population_by_region = df_new_group['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df_new_group['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    # raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [109]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_by_area():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # select 2018 data
    nhl_df = nhl_df[nhl_df['year']==2018]
    # remove *
    nhl_df['team'] = nhl_df['team'].map(lambda x: x.strip('*'))
    # remove irrelevant rows
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Atlantic Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Metropolitan Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Central Division'].index, inplace=True)
    nhl_df.drop(nhl_df[nhl_df['team'] == 'Pacific Division'].index, inplace=True)

    # create new column, city, and rename the column
    nhl_df['city'] = nhl_df['team'].apply(lambda x: x.split()[0])
    nhl_df['city'].replace({'New': 'New York City', 'St.': 'St. Louis', 'Vegas': 'Las Vegas', 'San': 'San Francisco Bay Area',
                       'Los': 'Los Angeles', 'Tempa': 'Tampa Bay Area', 'Florida': 'Miami–Fort Lauderdale', 'Washington': 'Washington, D.C.',
                       'Carolina': 'Raleigh', 'Minnesota': 'Minneapolis–Saint Paul', 'Colorado': 'Denver', 'Dallas': 'Dallas–Fort Worth',
                       'Anaheim': 'Los Angeles', 'Arizona': 'Phoenix', 'Tampa': 'Tampa Bay Area'}, inplace=True)
    # reformat data type for column, W, L, Population
    nhl_df['W'] = pd.to_numeric(nhl_df['W'])
    nhl_df['L'] = pd.to_numeric(nhl_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nhl_df, cities, left_on='city', right_on='Metropolitan area', how='inner')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    return df_new_group

def nba_by_area():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # select 2018 data
    nba_df = nba_df[nba_df['year']==2018]
    # remove irrelevant rows
    nba_df.drop(nba_df[nba_df['team'] == 'Atlantic Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Southeast Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Central Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Northwest Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Pacific Division'].index, inplace=True)
    nba_df.drop(nba_df[nba_df['team'] == 'Southwest Division'].index, inplace=True)

    # create new column, city, and rename the column
    nba_df['city'] = nba_df['team'].apply(lambda x: x.split()[0])
    # replace city name "new"
    nba_df['city'].replace({'San': 'San Antonio', 'Los': 'Los Angeles', 'Dallas': 'Dallas–Fort Worth', 'Indiana': 'Indianapolis',
                       'Miami': 'Miami–Fort Lauderdale', 'Washington': 'Washington, D.C.', 'Brooklyn': 'New York City',
                       'Golden': 'San Francisco Bay Area', 'Oklahoma': 'Oklahoma City', 'Utah': 'Salt Lake City', 'Minnesota': 'Minneapolis–Saint Paul',
                       'NewYorkCity': 'New York City', 'NewOrleans': 'New Orleans'}, inplace=True)
    nba_df['city'][10] = 'New York City'
    nba_df['city'][20] = 'New Orleans'
    # reformat data type for column, W, L, Population
    nba_df['W'] = pd.to_numeric(nba_df['W'])
    nba_df['L'] = pd.to_numeric(nba_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nba_df, cities, left_on='city', right_on='Metropolitan area', how='inner')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    return df_new_group

def mlb_by_area(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df = mlb_df[mlb_df['year']==2018]
    # create new column, city, and rename the column
    mlb_df['city'] = mlb_df['team'].apply(lambda x: x.split()[0])
    # rename city name 'San'
    mlb_df.loc[mlb_df['city'] == 'San', 'city'] = ['San Francisco Bay Area', 'San Diego']
    mlb_df['city'].replace({'New': 'New York City', 'Los': 'Los Angeles', 'St.': 'St. Louis',
                       'Tampa': 'Tampa Bay Area', 'Minnesota': 'Minneapolis–Saint Paul', 'Kansas': 'Kansas City',
                       'Texas': 'Dallas–Fort Worth', 'Washington': 'Washington, D.C.', 'Miami': 'Miami–Fort Lauderdale',
                       'Colorado': 'Denver', 'Arizona': 'Phoenix', 'Oakland': 'San Francisco Bay Area'}, inplace=True)
    # reformat data type for column, W, L, Population
    mlb_df['W'] = pd.to_numeric(mlb_df['W'])
    mlb_df['L'] = pd.to_numeric(mlb_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(mlb_df, cities, left_on='city', right_on='Metropolitan area', how='inner')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    return df_new_group

def nfl_by_area(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df = nfl_df[nfl_df['year']==2018]
    # remove irrelevant rows
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC East'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC North'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC South'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'AFC West'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC East'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC North'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC South'].index, inplace=True)
    nfl_df.drop(nfl_df[nfl_df['team'] == 'NFC West'].index, inplace=True)
    # create new column, city, and rename the column
    nfl_df['city'] = nfl_df['team'].apply(lambda x: x.split()[0])
    # rename city name "New"
    nfl_df.loc[nfl_df['city']=='New', 'city'] = ['Boston', 'New York City', 'New York City', 'New Orleans']
    nfl_df['city'].replace({'Miami': 'Miami–Fort Lauderdale', 'Tennessee': 'Nashville', 'Kansas': 'Kansas City',
                       'Los': 'Los Angeles', 'Oakland': 'San Francisco Bay Area', 'Dallas': 'Dallas–Fort Worth',
                       'Washington': 'Washington, D.C.', 'Minnesota': 'Minneapolis–Saint Paul', 'Green': 'Green Bay',
                       'Carolina': 'Charlotte', 'Tampa': 'Tampa Bay Area', 'San': 'San Francisco Bay Area',
                       'Arizona': 'Phoenix'}, inplace=True)
    # reformat data type for column, W, L, Population
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    # merge nhl_df and cities
    df_new = pd.merge(nfl_df, cities, left_on='city', right_on='Metropolitan area', how='inner')
    df_new = df_new[['city','W','L','Metropolitan area','Population (2016 est.)[8]']]
    # groupby Metropolitan area
    df_new_group = df_new.groupby('city').mean().reset_index()
    # calculate w/(w+l)
    df_new_group['ratio'] = df_new_group['W']/(df_new_group['W']+df_new_group['L'])
    return df_new_group

df_nhl = nhl_by_area()
df_nba = nba_by_area()
df_mlb = mlb_by_area()
df_nfl = nfl_by_area()

# perform hypothesis test for each pair
nhl_nba = pd.merge(df_nhl, df_nba, on='city', how='inner')
pval_nhl_nba = stats.ttest_rel(nhl_nba['ratio_x'], nhl_nba['ratio_y'])[1]
nhl_mlb = pd.merge(df_nhl, df_mlb, on='city', how='inner')
pval_nhl_mlb = stats.ttest_rel(nhl_mlb['ratio_x'], nhl_mlb['ratio_y'])[1]
nhl_nfl = pd.merge(df_nhl, df_nfl, on='city', how='inner')
pval_nhl_nfl = stats.ttest_rel(nhl_nfl['ratio_x'], nhl_nfl['ratio_y'])[1]
nba_mlb = pd.merge(df_nba, df_mlb, on='city', how='inner')
pval_nba_mlb = stats.ttest_rel(nba_mlb['ratio_x'], nba_mlb['ratio_y'])[1]
nba_nfl = pd.merge(df_nba, df_nfl, on='city', how='inner')
pval_nba_nfl = stats.ttest_rel(nba_nfl['ratio_x'], nba_nfl['ratio_y'])[1]
mlb_nfl = pd.merge(df_mlb, df_nfl, on='city', how='inner')
pval_mlb_nfl = stats.ttest_rel(mlb_nfl['ratio_x'], mlb_nfl['ratio_y'])[1]

def sports_team_performance():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values['NFL']['NBA'] = pval_nba_nfl; p_values['NFL']['NHL'] = pval_nhl_nfl; p_values['NFL']['MLB'] = pval_mlb_nfl
    p_values['NBA']['NHL'] = pval_nhl_nba; p_values['NBA']['MLB'] = pval_nba_mlb; p_values['NHL']['MLB'] = pval_nhl_mlb
    p_values['NBA']['NFL'] = pval_nba_nfl; p_values['NHL']['NFL'] = pval_nhl_nfl; p_values['MLB']['NFL'] = pval_mlb_nfl
    p_values['NHL']['NBA'] = pval_nhl_nba; p_values['MLB']['NBA'] = pval_nba_mlb; p_values['MLB']['NHL'] = pval_nhl_mlb    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
